In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split,cross_val_score,StratifiedKFold,KFold
from sklearn.feature_selection import SelectKBest,f_classif,SelectFdr
from sklearn import svm
from sklearn import preprocessing
from matplotlib import pyplot as plt
from sklearn.preprocessing import normalize,RobustScaler,StandardScaler
from sklearn.cluster import KMeans
from lifelines import CoxPHFitter
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import backend
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import glorot_uniform,RandomUniform,Constant
from scipy.stats import spearmanr
import statistics

GDSC

In [ ]:
# load the genes from GDSC
genes = pd.read_csv("genes_GDSC_oxa.csv")
genes

IMPRESS

In [ ]:
# load GSE data expression
impress = pd.read_csv("cellNorm22cpm_input.csv")
impress = impress.set_index(['Unnamed: 0'])
impress

In [ ]:
# make the order of genes consistent to GDSC
impress = impress.reindex(columns = genes["Unnamed: 0.1"])
impress

In [ ]:
# load AUC
auc = pd.read_csv("tmz paper_screening data and survival data_IN_12Nov21.csv")
auc = auc.set_index(['GS.number'])
auc = auc.reindex(impress.index)
auc

In [ ]:
# AUC distribution
plt.hist(auc.AUC)

In [ ]:
# linear transform AUC into range [0,10]
auc_norm = auc.AUC
auc_norm = (auc_norm - auc_norm.max())/(auc_norm.max() - auc_norm.min())+1
auc_norm = auc_norm*10
plt.hist(auc_norm)

In [ ]:
# create a basic DL model

seed_num = 1000
init = RandomUniform(seed = seed_num)

def create_model(activation1 = "sigmoid",activation2="softplus",l2 = 0.001,l1=0.001,opt = tf.keras.optimizers.Adam(learning_rate= 0.0001)):
     
    model = Sequential()
    
    model.add(Dense(1000,activation= activation1,input_shape=(10232,),
                    kernel_regularizer=regularizers.l2(l2),
                    activity_regularizer=regularizers.l1(l1),
                    kernel_initializer= init))       
    
    model.add(Dropout(0.3))
    
    model.add(Dense(100, activation=activation2,kernel_initializer= init))
                    
    model.add(Dropout(0.1))
    
    model.add(Dense(1, activation= activation2))
        
    model.compile(loss='mean_squared_error',optimizer = opt,metrics=['mae']) #SGD
        
    return model

In [ ]:
# plot loss function
def plt_loss(train_loss,validate_loss,fold_no):
    plt.figure(figsize=(8, 8))
    #plt.subplot(5, 1, fold_no)
    plt.plot(train_loss, label='Training Loss')
    plt.plot(validate_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Loss')
    #plt.ylim([0,0.6])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

In [ ]:
# load pre-trained HGCC weights
pre_model = create_model()
pre_model.load_weights("HGCC_model_only.h5")

In [ ]:
# normalize and predict GSE AUC with the pre-trained weights without fine-tune
Standard = StandardScaler()
impress_norm = Standard.fit_transform(impress)
pred_auc = pre_model.predict(impress_norm)

In [ ]:
# correlation
spearmanr(auc_norm,pred_auc)

In [ ]:
# create a DL model and load pre-trained weights on the first layer
def create_model2(activation1 = "sigmoid",activation2 = "softplus",l2 = 0.0001,l1=0.0001,opt = tf.keras.optimizers.Adam(learning_rate= 0.0001)):
    
    model = Sequential()
    
    model.add(Dense(1000,activation= activation1,input_shape=(10232,),
                    kernel_regularizer=regularizers.l2(l2),
                    activity_regularizer=regularizers.l1(l1),
                    weights = [pre_model.layers[0].get_weights()[0],pre_model.layers[0].get_weights()[1]]))            
    
    model.add(Dropout(0.3))
    
    model.add(Dense(100, activation=activation2,
                    kernel_initializer=init
                   )
             )
                    
    model.add(Dropout(0.1))
    
    model.add(Dense(1, activation= activation2, kernel_initializer=init))
        
    model.compile(loss='mse',
                  optimizer = opt,
                  metrics=['mean_absolute_error'])
        
    return model

In [ ]:
# use the same partition on the dataset to ensure a fair evaluation
train_test_index = pd.read_csv("train_test_index_sort.txt", sep="[", header=None)

line = 0

train_index = [0] * 10
test_index = [0] * 10

for seed in range(0,10):
    
    train_index[seed] = [0] * 3
    test_index[seed] = [0] * 3
    
    for fold in range(0,3):
        
        # get index train
        train_index[seed][fold] = train_test_index[1][line].split()
        # string to int
        train_index[seed][fold] = [eval(i) for i in train_index[seed][fold]]
        
                # get index train
        test_index[seed][fold] = train_test_index[2][line].split()
        # string to int
        test_index[seed][fold] = [eval(i) for i in test_index[seed][fold]]
        
        line += 1

train_index

In [ ]:
##### 3-fold CV to evaluate the model performance ####

# initiate the list of results
overall_cor_res = []

# 10 partitions
for k in range(0,10):
    
    exp_train = [0]*3
    exp_test = [0]*3
    auc_train = [0]*3
    auc_test = [0]*3
    pred_res = []
    # plot loss
    loss = []
    val_loss = []
    mse = []
    val_mse = []
    
    # 3-fold CV
    for fold_no in range(0,3):
    
        # split dataset
        exp_train[fold_no], exp_test[fold_no] = impress.iloc[train_index[k][fold_no],:], impress.iloc[test_index[k][fold_no],:]
        auc_train[fold_no], auc_test[fold_no] = auc_norm[train_index[k][fold_no]], auc_norm[test_index[k][fold_no]]
    
    
        # normalize
        exp_train[fold_no] = Standard.fit_transform(exp_train[fold_no])
        exp_test[fold_no] = Standard.fit_transform(exp_test[fold_no])
    
        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {(fold_no+1)}, seed {(k+1)} ...')
    
        # Create a basic model instance
        pred_model = create_model2(l2 = 0.0001,l1=0.0001,opt = tf.keras.optimizers.Adam(lr = 0.0001))
    
        refine_model = pred_model.fit(x=exp_train[fold_no], y=auc_train[fold_no], epochs=50, batch_size=32,
                                  validation_data=(exp_test[fold_no],auc_test[fold_no])
                                 )
        

    
        # predict and evaluate
        pred = pred_model.predict(exp_test[fold_no])
        pred = pd.DataFrame(pred)
        pred_res.append(pred)
        test = auc_test[fold_no].reset_index(drop = True)
        
    
        # calculate Spearman's correlation
        corr, _ = spearmanr(test, pred[0])
        print('Spearman correlation: %.3f' % corr)
    
        plt.figure(figsize=(8, 8))
        plt.scatter(test,pred[0])
        plt.plot([0, 10], [0, 10], 'k-', lw=2)
    
        # plot loss train and validate
        loss.append(refine_model.history['loss'])
        val_loss.append(refine_model.history['val_loss'])
        plt_loss(loss[fold_no],val_loss[fold_no],fold_no+1)
    
        # plot acc train and validate
        mse.append(refine_model.history['mean_absolute_error'])
        val_mse.append(refine_model.history['val_mean_absolute_error'])
        plt_loss(mse[fold_no],val_mse[fold_no],fold_no+1)


    
    # test_all
    test_all = np.concatenate([auc_test[0],auc_test[1],auc_test[2]]
    # pred_all 
    pred_all = np.concatenate([pred_res[0],pred_res[1],pred_res[2]])
    overall_cor, _ = spearmanr(test_all, pred_all)
    overall_cor_res.append(overall_cor)
    print('Overall spearman correlation: %.3f' % overall_cor)
    
